<a href="https://colab.research.google.com/github/lucasnbd/1TSCG-DbSuporte/blob/main/Checkpoint_2RM552092.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Biblioteca

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

df = pd.read_csv('/content/drive/MyDrive/aaa/Gastos_Publicidade_MelhoresCompras 1 (1).csv')

In [ ]:
df.head()

,Ano,Mes,Tipo de Midia,Gastos com Publicidade (em R$),Previsao Inicial de Aumento de Vendas (em mil unidades)
0,2022,1,Paginas web,87000,1000
1,2022,1,Redes sociais,120000,800
2,2022,1,TV,250000,1500
3,2022,1,Jornal,110000,730
4,2022,1,Revista,40000,360


### Transformação dos Dados

In [ ]:
df['Gastos com Publicidade (em R$)'] = df['Gastos com Publicidade (em R$)'].str.replace(',', '')

In [ ]:
df['Gastos com Publicidade (em R$)'] = pd.to_numeric(df['Gastos com Publicidade (em R$)'])
df['Previsao Inicial de Aumento de Vendas (em mil unidades)'] = pd.to_numeric(df['Previsao Inicial de Aumento de Vendas (em mil unidades)'])

In [ ]:
df['Gastos com Publicidade (em R$)'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: Gastos com Publicidade (em R$)
Non-Null Count  Dtype
--------------  -----
200 non-null    int64
dtypes: int64(1)
memory usage: 1.7 KB


### Regressões

In [ ]:
df_agrupado = df.groupby(['Ano', 'Mes', 'Tipo de Midia'])[['Gastos com Publicidade (em R$)', 'Previsao Inicial de Aumento de Vendas (em mil unidades)']].sum().reset_index()
df_agrupado = pd.get_dummies(df_agrupado, columns=['Tipo de Midia'])
df_agrupado

,Ano,Mes,Gastos com Publicidade (em R$),Previsao Inicial de Aumento de Vendas (em mil unidades),Tipo de Midia_Google,Tipo de Midia_Instagram,Tipo de Midia_Jornal,Tipo de Midia_Paginas web,Tipo de Midia_Radio,Tipo de Midia_Redes sociais,Tipo de Midia_Revista,Tipo de Midia_TV
0,2022,1,110000,730,False,False,True,False,False,False,False,False
1,2022,1,87000,1000,False,False,False,True,False,False,False,False
2,2022,1,75000,450,False,False,False,False,True,False,False,False
3,2022,1,120000,800,False,False,False,False,False,True,False,False
4,2022,1,40000,360,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
193,2024,7,231952,3418,False,False,False,True,False,False,False,False
194,2024,7,134288,1504,False,False,False,False,True,False,False,False
195,2024,7,308000,2951,False,False,False,False,False,True,False,False
196,2024,7,145152,1638,False,False,False,False,False,False,True,False


In [ ]:
df_agrupado.columns

Index(['Ano', 'Mes', 'Gastos com Publicidade (em R$)',
       'Previsao Inicial de Aumento de Vendas (em mil unidades)',
       'Tipo de Midia_Google', 'Tipo de Midia_Instagram',
       'Tipo de Midia_Jornal', 'Tipo de Midia_Paginas web',
       'Tipo de Midia_Radio', 'Tipo de Midia_Redes sociais',
       'Tipo de Midia_Revista', 'Tipo de Midia_TV'],
      dtype='object')

In [ ]:
#Categorizando as variáveis
X = df_agrupado[['Ano', 'Mes', 'Gastos com Publicidade (em R$)',
       'Tipo de Midia_Google', 'Tipo de Midia_Instagram',
       'Tipo de Midia_Jornal', 'Tipo de Midia_Paginas web',
       'Tipo de Midia_Radio', 'Tipo de Midia_Redes sociais',
       'Tipo de Midia_Revista', 'Tipo de Midia_TV']]
Y = df_agrupado['Previsao Inicial de Aumento de Vendas (em mil unidades)']

X_train, X_test, Y_train, Y_test = train_test_split(X , Y, test_size=0.2, random_state=50)

model_linear = LinearRegression()
model_linear.fit(X_train, Y_train)

# Criando e treinando o modelo de regressão polinomial
poly_features = PolynomialFeatures(degree=3)
X_poly_train = poly_features.fit_transform(X_train)
model_poly = LinearRegression()
model_poly.fit(X_poly_train, Y_train)

# Testando o R² da regressão linear usando bases de teste
r_squared_linear = model_linear.score(X_test, Y_test)

# Teste de hipóteses para a regressão linear
test_score_linear = []
for i in range(1000):  # Repetição para amostragem bootstrap
    X_test_sample, i, Y_test_sample, i = train_test_split(X_test, Y_test, test_size=0.2, random_state=50)
    test_score_linear.append(model_linear.score(X_test_sample, Y_test_sample))

# Analisando os coeficientes da regressão linear
coeficiente = model_linear.coef_
categorias = X.columns
coef_df = pd.DataFrame({'Categoria': categorias, 'Coeficiente': coeficiente})

# Calculando o impacto nas vendas para um real investido em cada categoria
coef_df['Aumento de 1 real na venda'] = coef_df['Coeficiente']

# Melhor e pior retorno
melhor_categoria = coef_df.loc[coef_df['Coeficiente'].idxmax()]
pior_categoria = coef_df.loc[coef_df['Coeficiente'].idxmin()]

# Testando o R² da regressão polinomial usando bases de teste
r_squared_poly = model_poly.score(poly_features.transform(X_test), Y_test)

# Teste de hipóteses para a regressão polinomial
test_score_poly = []
for i in range(1000):  # Repetição para amostragem bootstrap
    X_test_poly_sample = poly_features.transform(X_test)
    X_test_poly_sample, i, Y_test_sample, i = train_test_split(X_test_poly_sample, Y_test, test_size=0.2, random_state=50)
    test_score_poly.append(model_poly.score(X_test_poly_sample, Y_test_sample))

# Sua regressão linear obteve um R² de pelo menos 70% (1 ponto)
# Sua regressão linear obteve um R ² de pelo menos 90% (1 ponto)
print("Regressão Linear R²:", r_squared_linear)

print("Regressão Polinomial R²:", r_squared_poly)

print("\nCoeficientes da regressão linear e impacto no aumento de vendas por 1 unidade investida em cada categoria:")
print(coef_df)

# 7. Qual categoria tem o melhor retorno? E o pior retorno? (2 pontos)
print("\nCategoria com o melhor retorno:")
print(melhor_categoria)
print("\nCategoria com o pior retorno:")
print(pior_categoria)

Regressão Linear R²: 0.9325267815821656
Regressão Polinomial R²: 0.9846098368500484

Coeficientes da regressão linear e impacto no aumento de vendas por 1 unidade investida em cada categoria:
                         Categoria  Coeficiente  Aumento de 1 real na venda
0                              Ano   255.070967                  255.070967
1                              Mes   115.974383                  115.974383
2   Gastos com Publicidade (em R$)     0.000259                    0.000259
3             Tipo de Midia_Google  1989.900890                 1989.900890
4          Tipo de Midia_Instagram  2677.303829                 2677.303829
5             Tipo de Midia_Jornal -1007.132622                -1007.132622
6        Tipo de Midia_Paginas web  -283.996197                 -283.996197
7              Tipo de Midia_Radio -1258.557546                -1258.557546
8      Tipo de Midia_Redes sociais  -474.107565                 -474.107565
9            Tipo de Midia_Revista -1340.203237 